In [1]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [12]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in rude, arrogant language.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

In [17]:
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

In [20]:
model = init_chat_model(
    "claude-haiku-4-5-20251001",
    temperature=1.0,
    timeout=10,
    max_tokens=1000
)

In [21]:
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A condescending response (always required)
    response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

In [24]:


checkpointer = InMemorySaver()

In [26]:
agent = create_agent(
    model = model,
    system_prompt=SYSTEM_PROMPT,
    tools = [get_user_location, get_weather_for_location],
    context_schema = Context,
    response_format = ToolStrategy(ResponseFormat),
    checkpointer = checkpointer
)

config = {"configurable": {"thread_id": "1"}}

In [27]:
prompt = {"messages": [{"role": "user", "content": "what is the weather outside?"}]}

In [28]:
response = agent.invoke(
    prompt,
    config = config,
    context = Context(user_id = "1")
)

In [29]:
response

{'messages': [HumanMessage(content='what is the weather outside?', additional_kwargs={}, response_metadata={}, id='164d8339-2d99-47df-8fff-26d9f1ab5732'),
  AIMessage(content=[{'id': 'toolu_019Kbp99zxTh7e8nuE8SS982', 'input': {}, 'name': 'get_user_location', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01HWymE2MCv9AcrkuyyWrmp9', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 893, 'output_tokens': 23, 'server_tool_use': None, 'service_tier': 'standard', 'inference_geo': 'not_available'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--019c2f78-c903-7e50-be73-c387805c5f06-0', tool_calls=[{'name': 'get_user_location', 'args': {}, 'id': 'toolu_019Kbp99zxTh7e8nuE8SS982', 'type': 'tool_call'}], invalid_tool_calls=

In [31]:
response = agent.invoke(
    prompt,
    config = config,
    context = Context(user_id = "69")
)

In [35]:
for msg in response['messages']:
    msg.pretty_print()

================================ Human Message =================================

what is the weather outside?
================================== Ai Message ==================================

[{'id': 'toolu_019Kbp99zxTh7e8nuE8SS982', 'input': {}, 'name': 'get_user_location', 'type': 'tool_use'}]
Tool Calls:
  get_user_location (toolu_019Kbp99zxTh7e8nuE8SS982)
 Call ID: toolu_019Kbp99zxTh7e8nuE8SS982
  Args:
================================= Tool Message =================================
Name: get_user_location

Florida
================================== Ai Message ==================================

[{'id': 'toolu_01N7tEdEiyWCnTX66BpR5VwZ', 'input': {'city': 'Florida'}, 'name': 'get_weather_for_location', 'type': 'tool_use'}]
Tool Calls:
  get_weather_for_location (toolu_01N7tEdEiyWCnTX66BpR5VwZ)
 Call ID: toolu_01N7tEdEiyWCnTX66BpR5VwZ
  Args:
    city: Florida
================================= Tool Message =================================
Name: get_weather_for_location

It's always